In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
path = r'D:\Program Projects\Python Projects\temp\相似人群拓展\input\2018-Tencent-data\data_tencent'


In [ ]:
os.listdir(path)

In [ ]:
train=pd.read_csv(path+'/train.csv')
test=pd.read_csv(path+'/test1.csv')

In [ ]:
# aid：广告
# uid：用户
# label：是否种子标签
# 就是判断广告用户是否是核心种子
train

In [ ]:
# 检查缺失情况
assert train.isnull().sum().sum() == 0
assert test.isnull().sum().sum() == 0
print((train['label']==1).astype(int).mean())

# 训练集中种子用户达到4.8%

In [ ]:
# 双主键问题，看训练集和测试集中的用户和广告id是否重复了
# 检查训练集与测试集的uid与aid重合情况
def check_duplicate(feature):
    print('duplicate check of ' + feature)
    train_unique = train[feature].unique().tolist()
    test_unique = test[feature].unique().tolist()
    all_unique = list(set(train_unique + test_unique))
    print('train_unique', len(train_unique))
    print('test1_unique', len(test_unique))
    print('all_unique', len(all_unique))
    print('duplicate', len(train_unique) + len(test_unique) - len(all_unique))
    print('\n')
    return 
check_duplicate('uid')
check_duplicate('aid')
# 结论：可以看出测试集的用户只有不到18%出现在训练集中，而aid与训练集中出现的全部一样
# 

In [ ]:
train_unique = train[['uid','aid']].drop_duplicates().shape[0]
test_unique = test[['uid','aid']].drop_duplicates().shape[0]
all_unique = test[['uid','aid']]._append(train[['uid','aid']]).drop_duplicates().shape[0]
all_unique==train_unique+test_unique
# 说明双索引下，训练集测试集没有重复的

In [ ]:
(train["aid"].value_counts()/train.shape[0]).sort_index().plot()
(test["aid"].value_counts()/test.shape[0]).sort_index().plot()
plt.xlabel('aid')
plt.legend(['train','test1'])
plt.ylabel('ratio')
plt.show()
# 目测看起来分布一致，满足一致分布的情况下，可以继续进行训练

In [ ]:
# 读取用户信息
use_cols = ['uid','age','gender','marriageStatus','education','consumptionAbility','LBS','interest2','interest5','kw2']
ad_feature=pd.read_csv(r'D:\Program Projects\Python Projects\temp\相似人群拓展\input\2018-Tencent-data\data_tencent\adFeature.csv')
user_feature = pd.read_csv(r'D:\Program Projects\Python Projects\temp\相似人群拓展\input\2018-Tencent-data\data_tencent\userFeature.csv')



In [ ]:
ad_feature
user_feature.head()

In [ ]:
# 缺失值状况
user_feature.isnull().sum()

In [ ]:
# 缺失值比例
(user_feature.isnull().sum()/user_feature.shape[0]).plot.bar(figsize=(20,5))
plt.xlabel('feature')
plt.ylabel('null_ratio')
plt.show()

In [ ]:
# 方便参赛者快速理解并且跑通demo起见，对训练集与测试集进行随机采样，使得大数据问题转换为小数据
train = train.sample(frac=0.01, random_state=2020).reset_index(drop=True)
test = test.sample(frac=0.01, random_state=2020).reset_index(drop=True)
test['label'] = -2

In [ ]:
# 从中取出现有训练集与测试集的用户信息
user_feature = pd.merge(train._append(test), user_feature, how='left', on='uid').reset_index(drop=True)
user_feature.shape

In [ ]:
# 拼接广告信息
data = pd.merge(user_feature, ad_feature, how='left', on='aid')
del user_feature

In [ ]:
data.head()

In [ ]:
del train,test

In [ ]:
data['label'].value_counts()

In [ ]:
# 进行标签的转换，方便区分训练集与测试集
data['label'].replace(-1, 0, inplace=True)
data['label'].replace(-2, -1, inplace=True)
data['label'].value_counts()

In [ ]:
# 根据特征的单变量、多变量属性对特征类别进行区分
# 进行后续需要使用到的训练集、测试集以及测试集标签生成
train = data[data['label']>=0].reset_index(drop=True)
test = data[data['label']==-1].reset_index(drop=True)
del data

In [ ]:
# 根据字段类型划分
cols = train.columns.tolist()
cols.sort()
se = train[cols].dtypes
se

In [ ]:


# 多变量文本特征
text_features = se[se=='object'].index.tolist()

# 单变量特征
discrete_features = se[se!='object'].index.tolist()

discrete_features.remove('aid')
discrete_features.remove('uid')
discrete_features.remove('label')

In [ ]:
discrete_features
# 引入直接支持多变量文本特征的Catboost来建模

In [ ]:
text_features = ['appIdAction', 'appIdInstall', 'ct', 'interest1',
 'interest2', 'interest3', 'interest4', 'interest5', 'kw1',
 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1', 'topic2', 'topic3']

cat_features = ['LBS', 'adCategoryId', 'advertiserId', 'age', 'campaignId',
 'carrier', 'consumptionAbility', 'creativeId', 'creativeSize', 'education',
 'gender', 'house', 'productId', 'productType', 'aid', 'uid']
train[text_features] = train[text_features].fillna('00').astype(str)
test[text_features] = test[text_features].fillna('00').astype(str)
train[cat_features] = train[cat_features].fillna('00').astype(str)
test[cat_features] = test[cat_features].fillna('00').astype(str)

In [ ]:
# 将拼接好的表格进行存储
train.to_csv(path+'/train_sample.csv', index=False)
test.to_csv(path+'/test_sample.csv', index=False)

# test_truth.to_csv('preprocess/test_truth_sample.csv', index=False)

## 特征工程

In [1]:
import pandas as pd
path = r'D:\Program Projects\Python Projects\temp\相似人群拓展\input\2018-Tencent-data\data_tencent'
train = pd.read_csv(path+'/train_sample.csv')
test = pd.read_csv(path+'/test_sample.csv')

text_features = ['appIdAction', 'appIdInstall', 'ct', 'interest1',
 'interest2', 'interest3', 'interest4', 'interest5', 'kw1',
 'kw2', 'kw3', 'marriageStatus', 'os', 'topic1', 'topic2', 'topic3']
cat_features = ['LBS', 'adCategoryId', 'advertiserId', 'age', 'campaignId',
 'carrier', 'consumptionAbility', 'creativeId', 'creativeSize', 'education',
 'gender', 'house', 'productId', 'productType']

# truth_test = pd.read_csv('preprocess/test_truth_sample.csv')['label'].values

train_df = pd.DataFrame()
test_df = pd.DataFrame()

In [2]:
train["interest1"]

0                                  70 118 28 8 59 49 36 11
1                      70 118 76 75 8 29 59 49 42 35 36 11
2                                                    36 11
3                                              70 76 36 11
4        93 70 77 37 98 4 75 8 29 49 6 42 73 46 35 56 3...
                               ...                        
87983                            70 109 76 75 29 119 36 11
87984                                    70 12 76 88 36 11
87985    93 70 37 12 47 28 106 59 49 6 42 73 46 22 57 9...
87986                                                   00
87987       93 70 77 37 109 75 4 76 29 6 39 46 60 71 36 11
Name: interest1, Length: 87988, dtype: object

In [3]:
train[cat_features]

,LBS,adCategoryId,advertiserId,age,campaignId,carrier,consumptionAbility,creativeId,creativeSize,education,gender,house,productId,productType
0,612.0,140,11195,2,19215,0,0,1755470,53,7,1,0.0,0,4
1,458.0,13,1082,1,49189,1,1,333622,42,7,1,0.0,0,6
2,705.0,21,9106,5,662422,1,1,1354071,79,7,2,0.0,0,4
3,309.0,24,6946,2,296367,2,1,520004,59,1,1,1.0,3794,11
4,779.0,24,702,4,15634,2,1,480095,35,6,1,0.0,5615,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,686.0,10,452,5,50305,1,1,1187573,35,7,1,1.0,7992,11
87984,434.0,10,6841,2,159118,1,1,1758880,42,7,1,0.0,3733,11
87985,346.0,27,8864,5,90700,2,1,469197,22,4,1,0.0,113,9
87986,287.0,21,18621,2,154634,1,1,981822,35,2,2,1.0,0,4


In [4]:
se = train['LBS']._append(test['LBS']).value_counts()
se

LBS
458.0    3520
112.0    3202
809.0    3159
275.0    2794
346.0    2784
         ... 
381.0       1
661.0       1
850.0       1
777.0       1
461.0       1
Name: count, Length: 475, dtype: int64

In [5]:
# exposure
## 一维曝光度
for feature in cat_features:
    se = train[feature]._append(test[feature]).value_counts()
    train_df['exposure_' + feature] = train[feature].map(se).fillna(0).astype(int).values
    test_df['exposure_' + feature] = test[feature].map(se).fillna(0).astype(int).values

In [6]:

    
## 二维曝光度
n = len(cat_features)
for i in range(n-1):
    for j in range(i+1, n):
        col_name = "exposure_"+cat_features[i]+"_and_"+cat_features[j]
        cols = [cat_features[i],cat_features[j]]
        stat = train._append(test).groupby(cols).size().reset_index()
        stat.columns = cols + [col_name]
        
        train_df[col_name] = pd.merge(train[cols],stat,how='left',on=cols)[col_name].fillna(0).astype(int).values
        test_df[col_name] = pd.merge(test[cols],stat,how='left',on=cols)[col_name].fillna(0).astype(int).values
# nunique
num = 0
for i in range(n):
    for j in range(n):
        if i!=j:
            col_name = "nunique_"+cat_features[j]+"_in_"+cat_features[i]
            se = train._append(test).groupby([cat_features[i]])[cat_features[j]].value_counts()
            result = se.groupby(level=cat_features[i]).sum()
            
            train_df[col_name] = (train[cat_features[i]].map(se)).fillna(value=0).values
            test_df[col_name] = (test[cat_features[i]].map(se)).fillna(value=0).values

# ratio
for i in range(n-1):
    for j in range(i+1, n):
            col_both = "exposure_"+cat_features[i]+"_and_"+cat_features[j]
            
            col_one = "exposure_"+cat_features[j]
            col_name = "ratio_exposure_"+cat_features[i]+"_in_"+cat_features[j]
            train_df[col_name] = train_df[col_both]/train_df[col_one]
            test_df[col_name] = test_df[col_both]/test_df[col_one]
            
            col_one = "exposure_"+cat_features[i]
            col_name = "ratio_exposure_"+cat_features[j]+"_in_"+cat_features[i]
            train_df[col_name] = train_df[col_both]/train_df[col_one]
            test_df[col_name] = test_df[col_both]/test_df[col_one]

C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\1949013645.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[col_name] = pd.merge(train[cols],stat,how='left',on=cols)[col_name].fillna(0).astype(int).values
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\1949013645.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[col_name] = pd.merge(test[cols],stat,how='left',on=cols)[col_name].fillna(0).astype(int).values
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\1949013645.py:10: PerformanceWarning: DataFram

## 业务特征

In [7]:
## 每个离散字段不同取值得标签分布比例，在本次竞赛中就是点击率
# ctr (进阶版本可有贝叶斯平滑）
n_parts = 5
train['part'] = (pd.Series(train.index)%n_parts).values

## 一阶ctr
for co in cat_features:
    col_name = 'ctr_of_'+co
    ctr_train = pd.Series(dtype=float)
    ctr_test = pd.Series(0, index=test.index.tolist())
    for i in range(n_parts):
        se = train[train['part']!=i].groupby(co)['label'].mean()
        ctr_train = ctr_train._append(train[train['part']==i][co].map(se))
        ctr_test += test[co].map(se)
    
    train_df[col_name] = ctr_train.sort_index().fillna(-1).values
    test_df[col_name] = (ctr_test/5).fillna(-1).values

## 二阶ctr
for i in range(n-1):
    for j in range(i+1, n):
        col_name = 'ctr_of_'+cat_features[i]+"_and_"+cat_features[j]
        cols = [cat_features[i], cat_features[j]]
        ctr_train = pd.Series(dtype=float)
        ctr_test = pd.Series(0, index=test.index.tolist())

        for k in range(n_parts):
            stat = train[train['part']!=k].groupby(cols)['label'].mean().reset_index()
            stat.columns = cols + [col_name]
            
            se = pd.merge(train[train['part']==k][cols],stat,how='left',on=cols)[col_name]
            ctr_train = ctr_train._append(pd.Series(se.values, index=train[train['part']==k].index.tolist()))
            ctr_test += pd.merge(test[cols],stat,how='left',on=cols)[col_name]
        train_df[col_name] = ctr_train.sort_index().fillna(-1).values
        test_df[col_name] = (ctr_test/5).fillna(-1).values

C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\3563060209.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df[col_name] = ctr_train.sort_index().fillna(-1).values
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\3563060209.py:17: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df[col_name] = (ctr_test/5).fillna(-1).values
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\3563060209.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which h

# 文本特征

In [8]:
from scipy import sparse
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder

In [9]:
text_features

['appIdAction',
 'appIdInstall',
 'ct',
 'interest1',
 'interest2',
 'interest3',
 'interest4',
 'interest5',
 'kw1',
 'kw2',
 'kw3',
 'marriageStatus',
 'os',
 'topic1',
 'topic2',
 'topic3']

In [10]:
train["interest1"]

0                                  70 118 28 8 59 49 36 11
1                      70 118 76 75 8 29 59 49 42 35 36 11
2                                                    36 11
3                                              70 76 36 11
4        93 70 77 37 98 4 75 8 29 49 6 42 73 46 35 56 3...
                               ...                        
87983                            70 109 76 75 29 119 36 11
87984                                    70 12 76 88 36 11
87985    93 70 37 12 47 28 106 59 49 6 42 73 46 22 57 9...
87986                                                   00
87987       93 70 77 37 109 75 4 76 29 6 39 46 60 71 36 11
Name: interest1, Length: 87988, dtype: object

In [11]:
def change_stop_words(se):
    return se.apply(lambda x: 'stop '+x)


# stop words
for feature in text_features:
    train[feature] = change_stop_words(train[feature])
    test[feature] = change_stop_words(test[feature])

In [12]:
train["interest1"]

0                             stop 70 118 28 8 59 49 36 11
1                 stop 70 118 76 75 8 29 59 49 42 35 36 11
2                                               stop 36 11
3                                         stop 70 76 36 11
4        stop 93 70 77 37 98 4 75 8 29 49 6 42 73 46 35...
                               ...                        
87983                       stop 70 109 76 75 29 119 36 11
87984                               stop 70 12 76 88 36 11
87985    stop 93 70 37 12 47 28 106 59 49 6 42 73 46 22...
87986                                              stop 00
87987    stop 93 70 77 37 109 75 4 76 29 6 39 46 60 71 ...
Name: interest1, Length: 87988, dtype: object

In [13]:
cat_features

['LBS',
 'adCategoryId',
 'advertiserId',
 'age',
 'campaignId',
 'carrier',
 'consumptionAbility',
 'creativeId',
 'creativeSize',
 'education',
 'gender',
 'house',
 'productId',
 'productType']

In [14]:
train["LBS"]

0        612.0
1        458.0
2        705.0
3        309.0
4        779.0
         ...  
87983    686.0
87984    434.0
87985    346.0
87986    287.0
87987    101.0
Name: LBS, Length: 87988, dtype: float64

In [15]:
# labelencoder
lab = LabelEncoder()
for feature in cat_features:
    lab.fit(train[feature]._append(test[feature]))
    
    train_df['labelencoder_'+feature] = lab.transform(train[feature])
    test_df['labelencoder_'+feature] = lab.transform(test[feature])

C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\329117642.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_df['labelencoder_'+feature] = lab.transform(train[feature])
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\329117642.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_df['labelencoder_'+feature] = lab.transform(test[feature])
C:\Users\yx140\AppData\Local\Temp\ipykernel_79548\329117642.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many tim

In [16]:
train_df['labelencoder_LBS']

0        293
1        221
2        337
3        146
4        375
        ... 
87983    325
87984    210
87985    166
87986    138
87987     49
Name: labelencoder_LBS, Length: 87988, dtype: int64

In [17]:

# onehot
train_sp = pd.DataFrame()
test_sp = pd.DataFrame()


ohe = OneHotEncoder()
for feature in cat_features:
    ohe.fit(train[feature]._append(test[feature]).values.reshape(-1, 1))
    
    arr = ohe.transform(train[feature].values.reshape(-1, 1))
    # 将独热编码的结果以稀疏矩阵形式拼接到 train_sp 中，以节省内存和计算资源。
    train_sp = sparse.hstack((train_sp, arr))
    
    arr = ohe.transform(test[feature].values.reshape(-1, 1))
    test_sp = sparse.hstack((test_sp, arr))

In [18]:
# countvectorizer
# 这段代码是对文本特征进行词频统计（Count Vectorization），
# 将文本数据转换为特征矩阵，并将结果添加到稀疏矩阵 train_sp 和 test_sp 中。
# 具体作用是将文本数据表示为词频向量，为模型提供结构化的数值输入。
cntv=CountVectorizer()
for feature in text_features:
    cntv.fit(train[feature]._append(test[feature]))
    
    train_sp = sparse.hstack((train_sp, cntv.transform(train[feature])))
    test_sp = sparse.hstack((test_sp, cntv.transform(test[feature])))

In [19]:
train[text_features]
# 在 TF-IDF 或 CountVectorizer 中，停用词会被单独处理，它们的重要性通过 TF-IDF 计算时会降低权重。
# 这可以帮助模型在面对大量冗余信息时，更关注那些包含停用词的上下文。

,appIdAction,appIdInstall,ct,interest1,interest2,interest3,interest4,interest5,kw1,kw2,kw3,marriageStatus,os,topic1,topic2,topic3
0,stop 00,stop 00,stop 1 3,stop 70 118 28 8 59 49 36 11,stop 1 70 35,stop 00,stop 00,stop 00,stop 409571 4339 589925 605494 210691,stop 43582 73814 43977 18780 18185,stop 00,stop 13 10,stop 1,stop 8228 3292 4080 6794 7764,stop 8219 9592 1335 1352 6517,stop 00
1,stop 00,stop 00,stop 1 3,stop 70 118 76 75 8 29 59 49 42 35 36 11,stop 70 21 33 10,stop 00,stop 00,stop 92 37 50 6 76 27 75 46 62 123 59 60 93 26,stop 529348 107700 297041 715689 371925,stop 35605 90383 13916 55458 50239,stop 00,stop 13 10,stop 1,stop 7037 7811 6854 5038 8137,stop 8985 5927 8867 266 7569,stop 00
2,stop 00,stop 00,stop 1 3,stop 36 11,stop 73 46 74 72,stop 00,stop 00,stop 52 92 131 116 134 109 8 6 42 5 76 39 75 5...,stop 193810 36720 639355,stop 66915 33773 106194 70672 48624,stop 00,stop 10,stop 1,stop 9826 191 5836 3773 948,stop 5451 558 5083 8910 7621,stop 00
3,stop 00,stop 00,stop 3 1,stop 70 76 36 11,stop 00,stop 00,stop 00,stop 52 72 92 116 11 78 79 128 8 30 6 76 121 5...,stop 515969 615585 162804 703870 743380,stop 116369 85039 47888 56819 51859,stop 00,stop 10,stop 2,stop 6508 1390 5891 8786 2094,stop 3643 3349 7043 6058 1341,stop 00
4,stop 00,stop 00,stop 1 2 3,stop 93 70 77 37 98 4 75 8 29 49 6 42 73 46 35...,stop 24 52 21 30,stop 00,stop 00,stop 100 72 80 92 37 13 78 4 109 8 30 50 28 6 ...,stop 685851 37509 782594 11216 176009,stop 75000 101895 12142 11395 79112,stop 00,stop 11,stop 1,stop 6021 9826 8525 7684 5302,stop 4197 2570 6874 9047 9125,stop 00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,stop 00,stop 00,stop 3 1 4,stop 70 109 76 75 29 119 36 11,stop 9 33 76 23,stop 00,stop 00,stop 52 131 37 116 13 78 4 8 30 6 132 39 9 46 ...,stop 331786 4830 208572 41040 389712,stop 117548 17710 2095 53849 84734,stop 00,stop 10,stop 2,stop 2636 5311 5338 2549 6031,stop 945 2990 2703 2766 8673,stop 00
87984,stop 00,stop 00,stop 1 3 2,stop 70 12 76 88 36 11,stop 73 70 74 10 35,stop 00,stop 00,stop 72 92 116 101 13 47 1 78 71 109 8 30 28 1...,stop 8561 405681 767639 562294 32192,stop 43582 15571 49707 114283 88002,stop 00,stop 10,stop 1,stop 2030 604 8110 5579 7658,stop 1352 4745 982 8822 8880,stop 00
87985,stop 00,stop 00,stop 3 2 1 4,stop 93 70 37 12 47 28 106 59 49 6 42 73 46 22...,stop 24 79 73 31 46 6 19 13 30 44 72 29,stop 00,stop 00,stop 100 72 92 37 116 101 13 78 71 109 8 30 50...,stop 83743 518360 58458 700029 146637,stop 56438 99827 83281 5262 106104,stop 00,stop 11,stop 2,stop 2665 9826 941 7224 5997,stop 4431 6477 2920 122 9748,stop 00
87986,stop 00,stop 00,stop 1 4 3,stop 00,stop 42 73 1 46 70 6 21 74 37,stop 00,stop 00,stop 131 78 109 8 30 6 89 59 129 83,stop 00,stop 107549 17528 24659 14470 120667,stop 00,stop 10,stop 1,stop 00,stop 4739 5791 6457 7358 1564,stop 00


In [20]:
# tf-idf
tfd = TfidfVectorizer()
for feature in text_features:
    tfd.fit(train[feature]._append(test[feature]))
    
    train_sp = sparse.hstack((train_sp, tfd.transform(train[feature])))
    test_sp = sparse.hstack((test_sp, tfd.transform(test[feature])))

In [21]:
# 造个数据验证TfidfVectorizer的作用
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# 构造示例文本数据
train_data = ["the cat and the dog", "the quick brown fox", "dog and fox are friends"]
test_data = ["the cat is quick", "brown fox and dog"]

# 初始化 TfidfVectorizer
tfd = TfidfVectorizer()

# 拟合并转换训练集
train_tfidf = tfd.fit_transform(train_data)

# 转换测试集
test_tfidf = tfd.transform(test_data)

# 将结果转换为 DataFrame 便于展示
train_tfidf_df = pd.DataFrame(train_tfidf.toarray(), columns=tfd.get_feature_names_out())
test_tfidf_df = pd.DataFrame(test_tfidf.toarray(), columns=tfd.get_feature_names_out())

train_tfidf_df


,and,are,brown,cat,dog,fox,friends,quick,the
0,0.359700,0.00000,0.000000,0.472963,0.359700,0.000000,0.00000,0.000000,0.719401
1,0.000000,0.00000,0.562829,0.000000,0.000000,0.428046,0.00000,0.562829,0.428046
2,0.393511,0.51742,0.000000,0.000000,0.393511,0.393511,0.51742,0.000000,0.000000


In [22]:
test_tfidf_df

,and,are,brown,cat,dog,fox,friends,quick,the
0,0.000000,0.0,0.000000,0.622766,0.000000,0.000000,0.0,0.622766,0.47363
1,0.459854,0.0,0.604652,0.000000,0.459854,0.459854,0.0,0.000000,0.00000


In [23]:
train_sp.shape

(87988, 251523)

## 特征降维

In [25]:
# 稀疏特征降维 TruncatedSVD

# TruncatedSVD 是一种 降维算法，主要用于对 稀疏特征矩阵 进行低维表示。
# 在高维稀疏特征（例如由 TF-IDF 或 CountVectorizer 生成的矩阵）中，
# TruncatedSVD 可以提取最重要的信息，同时显著减少维度。


from sklearn.decomposition import TruncatedSVD
from scipy.sparse import random as sparse_random

# 迭代50次 降维到100个特征
svd = TruncatedSVD(n_components=100, n_iter=50, random_state=2020) 
svd.fit(sparse.vstack((train_sp, test_sp)))

cols = ['svd_'+str(k) for k in range(100)]

train_svd = pd.DataFrame(svd.transform(train_sp), columns = cols)
test_svd = pd.DataFrame(svd.transform(test_sp), columns = cols)

In [30]:
train_svd

,svd_0,svd_1,svd_2,svd_3,svd_4,svd_5,svd_6,svd_7,svd_8,svd_9,...,svd_90,svd_91,svd_92,svd_93,svd_94,svd_95,svd_96,svd_97,svd_98,svd_99
0,6.015539,2.104419,-0.164067,0.433407,-1.189694,0.049969,-0.197846,-1.526452,-0.302052,-0.338819,...,-0.027841,-0.097942,0.028644,-0.147175,-0.080100,0.101774,0.147739,0.261906,0.094165,-0.053741
1,6.735295,-0.261891,-0.242245,-0.030485,-0.344259,-0.131470,-0.047261,-0.936713,-0.168223,-0.806769,...,0.033854,-0.436057,0.155044,0.236996,-0.245245,-0.029235,0.247810,0.153201,0.089548,0.028287
2,6.541299,-0.153541,-1.913142,0.047606,-0.698782,0.708737,1.053949,0.005991,-0.488833,-0.477656,...,0.058190,-0.153191,0.008242,0.092964,-0.175672,-0.360838,0.082763,-0.373719,0.293583,0.091234
3,6.496131,0.334106,-1.605413,0.850955,2.256874,-0.049980,-0.694393,-0.069620,-0.545849,-1.484629,...,-0.049419,0.010267,-0.085544,0.313523,-0.122954,0.063868,0.087942,0.163877,0.368014,0.077935
4,7.812981,-2.362091,1.995630,-0.557136,0.860026,-0.739366,-0.265708,1.102979,-0.470898,0.557988,...,0.211865,-0.360781,0.233128,0.509409,-0.127072,0.088962,0.079112,-0.346054,-0.594294,-0.070130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,6.550897,-0.233029,-0.336556,-0.126108,1.453989,0.037043,0.428808,-0.312991,0.738040,-0.995342,...,0.060908,-0.054495,-0.310391,-0.018571,-0.279718,0.156157,0.273381,0.105913,0.034025,0.006896
87984,6.847381,-0.743862,-2.055528,-0.440724,1.235782,-0.336430,0.163765,-0.939977,-0.502598,-0.645120,...,0.075626,0.211550,-0.043963,-0.123683,-0.193367,0.065224,0.083160,0.149290,0.353257,-0.044345
87985,7.702174,-2.558863,0.267117,-0.906016,0.467589,-0.616725,0.432093,-0.046258,-0.235285,1.661293,...,-0.053034,0.261738,-0.749645,-0.001522,0.259549,0.276280,-0.539243,-0.143605,-0.274255,0.385411
87986,5.924332,1.163341,-2.340327,-1.383325,-0.449198,0.614586,1.651590,0.007881,-0.285196,0.880821,...,-0.087818,-0.058740,-0.008674,-0.314907,-0.064023,0.155914,-0.181233,-0.529420,-0.072228,-0.165841


# 存储特征

In [29]:
sparse.save_npz(path+"/train_sample_sparse.npz",train_sp)
sparse.save_npz(path+"/test_sample_sparse.npz",test_sp)

train_df.to_csv(path+"/train_sample_feature.csv", index=False)
test_df.to_csv(path+"/test_sample_feature.csv", index=False)

train_svd.to_csv(path+"/train_sample_svd.csv", index=False)
test_svd.to_csv(path+"/test_sample_svd.csv", index=False)

# 模型训练

In [ ]:
"""
feature + svd
"""
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import KFold
from hyperopt import hp, fmin, tpe
from numpy.random import RandomState
from sklearn.metrics import roc_auc_score


def params_append(params):
    """
    对参数进行额外赋值
    :param params:参数字典
    :return:更新后的参数字典
    """
    params['objective'] = 'binary'
    params['metric'] = 'auc'
    params['bagging_seed'] = 2020
    return params


def param_hyperopt(train):
    """
    采用hyperopt框架进行参数寻优
    :param train:训练集
    :return:
    """
    def hyperopt_objective(params):
        """
        超参调优的目标函数
        :param params:参数字典
        :return:越小越好，所以取auc对应的负值
        """
        params = params_append(params)
        print(params)
        res = lgb.cv(params, train_data, 1000,
                     nfold=2,
                     stratified=False,
                     shuffle=True,
                     metrics='auc',
                     early_stopping_rounds=20,
                     verbose_eval=False,
                     show_stdv=False,
                     categorical_feature=cat_features,
                     seed=2020)
        return -max(res['auc-mean'])
    label = 'label'
    features = train.columns.tolist()
    features.remove('label')
    cat_features= [feature for feature in features if 'labelencoder' in feature]
    train_data = lgb.Dataset(train[features], train[label], silent=True)
    params_space = {
        'learning_rate': hp.uniform('learning_rate', 1e-2, 5e-1),
        'bagging_fraction': hp.uniform('bagging_fraction', 0.5, 1),
        'feature_fraction': hp.uniform('feature_fraction', 0.5, 1),
        'num_leaves': hp.choice('num_leaves', list(range(10, 300, 10))),
        'reg_alpha': hp.randint('reg_alpha', 0, 10),
        'reg_lambda': hp.uniform('reg_lambda', 0, 10),
        'bagging_freq': hp.randint('bagging_freq', 1, 10),
        'min_child_samples': hp.choice('min_child_samples', list(range(1, 30, 5)))
    }
    params_best = fmin(
        hyperopt_objective,
        space=params_space,
        algo=tpe.suggest,
        max_evals=30,
        rstate=RandomState(2020))
    return params_best


def train_predict(train, test, params, debug):
    """

    :param train:
    :param test:
    :param params:
    :param debug:
    :return:
    """

    label = 'label'
    features = train.columns.tolist()
    features.remove(label)
    print(train.shape, test.shape)
    cat_features= [feature for feature in features if 'labelencoder' in feature]
    params = params_append(params)
    kf = KFold(n_splits=5, random_state=2020, shuffle=True)
    prediction_test = 0
    cv_score = []
    prediction_train = pd.Series()
    ESR = 30
    NBR = 10000
    VBE = 50
    for train_part_index, eval_index in kf.split(train[features], train[label]):
        # 模型训练
        train_part = lgb.Dataset(train[features].loc[train_part_index],
                                 train[label].loc[train_part_index])
        eval = lgb.Dataset(train[features].loc[eval_index],
                           train[label].loc[eval_index])
        bst = lgb.train(params, train_part, num_boost_round=NBR,
                        valid_sets=[train_part, eval],
                        valid_names=['train', 'valid'],categorical_feature=cat_features,
                        early_stopping_rounds=ESR, verbose_eval=VBE)
        prediction_test += bst.predict(test[features])
        prediction_train = prediction_train.append(pd.Series(bst.predict(train[features].loc[eval_index]),
                                                             index=eval_index))
        cv_score.append(bst.best_score['valid']['auc'])
    print(cv_score, sum(cv_score) / 5)
    NROWS = 10000 if debug else None
    truth_test = pd.read_csv('preprocess/test_truth_sample.csv', nrows=NROWS)['label'].values
    print(roc_auc_score(truth_test, prediction_test / 5))
    pd.Series(prediction_train.sort_index().values).to_csv("result/train_lightgbm.csv", index=False)
    pd.Series(prediction_test / 5).to_csv("result/test_lightgbm.csv", index=False)
    return


ModuleNotFoundError: No module named 'hyperopt'

In [35]:
def read_data(debug=True):
    """
    读取数据
    :param debug:代码调试
    :return:
    """
    print("read_data...")
    NROWS = 10000 if debug else None
    train_df = pd.read_csv(path + "/train_sample_feature.csv", nrows=NROWS)
    test_df = pd.read_csv(path + "/test_sample_feature.csv", nrows=NROWS)
    train_svd = pd.read_csv(path + "/train_sample_svd.csv", nrows=NROWS)
    test_svd = pd.read_csv(path + "/test_sample_svd.csv", nrows=NROWS)
    train = pd.concat([train_df, train_svd], axis=1)
    test = pd.concat([test_df, test_svd], axis=1)
    train['label'] = pd.read_csv(path + "/train_sample.csv", nrows=NROWS)['label'].values
    print("done")
    return train, test

In [36]:


if __name__ == "__main__":

    # 读取数据
    debug = False
    train, test = read_data(debug)

    # 进行参数寻优
    # best_clf = param_hyperopt(train)
    # print(best_clf)

    # # 寻优后设置参数
    # best_clf = {'bagging_fraction': 0.920269687818116, 'bagging_freq': 1,
    #             'feature_fraction': 0.6748121703594134, 'learning_rate': 0.2534183350764367,
    #             'min_child_samples': 3, 'num_leaves': 10, 'reg_alpha': 0,
    #             'reg_lambda': 9.986223861515665}

    # # 训练预测和结果输出
    # train_predict(train, test, best_clf, debug)

    # 模型效果
    # [0.6757995747895691, 0.6774535654447692, 0.6977578977587808, 0.6788931645556028, 0.6661883292824802]
    # 0.6186427592654342

read_data...
done


In [39]:
train

,exposure_LBS,exposure_adCategoryId,exposure_advertiserId,exposure_age,exposure_campaignId,exposure_carrier,exposure_consumptionAbility,exposure_creativeId,exposure_creativeSize,exposure_education,...,svd_91,svd_92,svd_93,svd_94,svd_95,svd_96,svd_97,svd_98,svd_99,label
0,280,3188,3188,25501,3188,30877,23634,2068,10882,39299,...,-0.097942,0.028644,-0.147175,-0.080100,0.101774,0.147739,0.261906,0.094165,-0.053741,0
1,3520,5861,5430,26048,110,53265,65486,110,3919,39299,...,-0.436057,0.155044,0.236996,-0.245245,-0.029235,0.247810,0.153201,0.089548,0.028287,0
2,1073,18808,6408,26029,1259,53265,65486,1259,9659,39299,...,-0.153191,0.008242,0.092964,-0.175672,-0.360838,0.082763,-0.373719,0.293583,0.091234,0
3,134,13415,9734,25501,9734,11119,65486,2659,21886,13248,...,0.010267,-0.085544,0.313523,-0.122954,0.063868,0.087942,0.163877,0.368014,0.077935,0
4,288,13415,5504,21106,523,11119,65486,314,23783,19068,...,-0.360781,0.233128,0.509409,-0.127072,0.088962,0.079112,-0.346054,-0.594294,-0.070130,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87983,542,14199,2582,26029,1723,53265,65486,1723,23783,39299,...,-0.054495,-0.310391,-0.018571,-0.279718,0.156157,0.273381,0.105913,0.034025,0.006896,0
87984,611,14199,212,25501,212,53265,65486,212,3919,39299,...,0.211550,-0.043963,-0.123683,-0.193367,0.065224,0.083160,0.149290,0.353257,-0.044345,0
87985,2784,13044,3200,26029,1716,11119,65486,1716,17497,1222,...,0.261738,-0.749645,-0.001522,0.259549,0.276280,-0.539243,-0.143605,-0.274255,0.385411,0
87986,261,18808,1474,25501,352,53265,65486,352,23783,27907,...,-0.058740,-0.008674,-0.314907,-0.064023,0.155914,-0.181233,-0.529420,-0.072228,-0.165841,0
